In [1]:
%pip install nltk -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import json
from pandas import json_normalize
import re
from nltk.tokenize import WordPunctTokenizer

#open json file
with open('result.json', encoding="utf8") as f:
    d = json.load(f)

#normalize it according to parent node
norm_msg = json_normalize(d['messages'])
msg_df = pd.DataFrame(norm_msg) #store it in a dataframe

In [2]:
msg_df_filtered = msg_df[msg_df.type=="message"] #filter only message type message 
msg_df_filtered = msg_df_filtered[["date","text","from"]] #filter important variables 
msg_df_filtered

,date,text,from
1,2021-11-11T15:43:50,👀👀👀,Adam Benayoun
2,2021-11-11T16:01:20,,Legendary Harry Pottre
3,2021-11-11T16:34:09,,The Owl Keeper
4,2021-11-11T16:42:09,,SAW | TheToadKeeper
5,2021-11-12T01:37:57,,Jinwoo
...,...,...,...
51932,2022-11-07T14:59:20,"Anyone you wish to use, no difference between ...",Nalma ~Will Never PM For Fund$~
51933,2022-11-07T15:08:38,thks very much for information,Oryan94
51934,2022-11-07T15:32:16,"""User can not log in"" this is the display mess...",Jany
51935,2022-11-07T15:36:55,Guys I just had a bug occurring on the android...,Gabriele


In [7]:
import json

with open('output.jsonl', 'w') as outfile:
    for entry in d:
        json.dump(entry, outfile)
        outfile.write('\n')

In [21]:
temp = msg_df_filtered #duplicate the df
temp['text'] = temp['text'].str.replace('[^A-Za-z0-9]+', " ", regex=True) #replace all non-alaphabet characters
temp = temp.dropna() #remove the na

In [22]:
common_word = pd.Series(' '.join(temp['text']).lower().split()).value_counts()[:100] #show top 100 words used
common_word = common_word.reset_index().rename(columns={"index":"Word", 0:"Count"}) #reset the index and rename the columns 

In [ ]:
%pip install plotly
%pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import plotly.express as px

fig = px.bar(common_word, x='Word', y='Count')
fig.show()

In [ ]:
messages_sent = temp['from'].value_counts()
messages_sent = messages_sent.reset_index().rename(columns={"index":"User", "from":"Count"})

fig2 = px.bar(messages_sent, x='User', y='Count')
fig2.show()

In [ ]:
replies = [temp.iloc[n,2] + " to " + temp.iloc[(n-1),2] for n in range(1,len(temp))] #list constructor to create replying list
replies.insert(0,"A to A") #insert an initial convo to make list same length as df

temp['Replies'] = replies #added list to df

replies_to = temp['Replies'].value_counts() #count
replies_to = replies_to.reset_index().rename(columns={"index":"Replies", "Replies":"Count"}) #reset index and rename

fig3 = px.bar(replies_to, x='Replies', y='Count')
fig3.show()

/tmp/ipykernel_12696/3428973302.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

